# test/Test 1

This notebook analyzes the matching decisions by the simulation.py and summarizes a few metrics

In [158]:
import pandas as pd
import numpy as np

In [159]:
recipients = pd.read_csv('data/test/test 1/test_recipients.csv',index_col=0,parse_dates=['date'])
recipients = recipients[recipients['qty']>0]
recipients

,rec_id,date,ppe,qty
0,R1,2020-04-10 13:08:00+00:00,gowns,100
1,R1,2020-04-10 13:08:00+00:00,gowns,100
2,R2,2020-04-10 13:08:00+00:00,gowns,200


In [160]:
donors = pd.read_csv('data/test/test 1/test_donors.csv',index_col=0,parse_dates=['date'])
donors

,don_id,date,ppe,qty,don_req_id
0,D1,2020-04-09 13:08:00+00:00,gowns,150,0
1,D1,2020-04-09 13:08:00+00:00,gowns,150,1
2,D2,2020-04-10 13:08:00+00:00,gowns,100,2


In [161]:
decisions = pd.read_csv('data/test/test 1/test_decisions.csv',index_col=0,parse_dates=['date'])
decisions

,date,don_id,rec_id,ppe,qty
0,2020-04-11 13:08:00+00:00,D1,R1,gowns,100
1,2020-04-11 13:08:00+00:00,D2,R1,gowns,50
2,2020-04-11 13:08:00+00:00,D2,R2,gowns,100
3,2020-04-12 13:08:00+00:00,D1,R1,gowns,50
4,2020-04-12 13:08:00+00:00,D2,R2,gowns,100


In [162]:
all_ppes = set(donors.ppe.unique())
all_ppes = all_ppes.union(set(recipients.ppe.unique()))

In [163]:
cur_recipients = recipients.copy()
cur_donors = donors.copy()

# Aggregated decisions at time 11

In [164]:
cur_decisions=decisions[decisions.date == decisions.date.min()]
cur_decisions

,date,don_id,rec_id,ppe,qty
0,2020-04-11 13:08:00+00:00,D1,R1,gowns,100
1,2020-04-11 13:08:00+00:00,D2,R1,gowns,50
2,2020-04-11 13:08:00+00:00,D2,R2,gowns,100


Now, based on these decisions, I need to get to granular decisions and matches

In [165]:
granular_decisions = pd.DataFrame(columns=['don_id','rec_id','ppe', 'date','qty','distance','holding time'])

In [166]:
cur_donors

,don_id,date,ppe,qty,don_req_id
0,D1,2020-04-09 13:08:00+00:00,gowns,150,0
1,D1,2020-04-09 13:08:00+00:00,gowns,150,1
2,D2,2020-04-10 13:08:00+00:00,gowns,100,2


In [167]:
cur_recipients

,rec_id,date,ppe,qty
0,R1,2020-04-10 13:08:00+00:00,gowns,100
1,R1,2020-04-10 13:08:00+00:00,gowns,100
2,R2,2020-04-10 13:08:00+00:00,gowns,200


Debug the first decision

In [168]:
# test
#cur_dec = cur_decisions.iloc[0] # just for test
cur_dec['qty'] = 185
cur_dec

ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


date      2020-04-11 13:08:00+00:00
don_id                           D1
rec_id                           R1
ppe                           gowns
qty                             185
Name: 0, dtype: object

In [169]:

don = cur_dec.don_id
rec = cur_dec.rec_id
ppe = cur_dec.ppe
dd = cur_dec.date
totremqty = cur_dec.qty
don_df = cur_donors[(cur_donors.don_id == don)&(cur_donors.ppe == ppe)].sort_values('date') # just this ppe and don rec
rec_df = cur_recipients[(cur_recipients.rec_id == rec)&(cur_recipients.ppe == ppe)].sort_values('date')

In [170]:
dilocx = 0
rilocx = 0
print(f'decision: ship {totremqty} from {don} to {rec}')
while totremqty > 0:
    drow = don_df.iloc[dilocx]
    rrow = rec_df.iloc[rilocx]
    dix = drow.name
    rix = rrow.name
    print ('========================================================')
    print(f'cur_donors = \n{cur_donors}\n cur_recipients = \n{cur_recipients}\n')
    print(f'remaining qty = {totremqty}. Donor row = [{drow.name,drow.don_id,drow.qty}]. Recipient row = [{rrow.name,rrow.rec_id,rrow.qty}].')
    shipped_qty = min(drow.qty,rrow.qty,totremqty)
    # make the granular dewcision of shipping
    granular_decisions.loc[len(granular_decisions)] = [drow.don_id,rrow.rec_id,ppe,dd,shipped_qty,-1,(dd-drow.date).days]
    print (f'Granular decisions: ship {shipped_qty} from {drow.don_id} to {rrow.rec_id}')
    # update quantities    
    totremqty-=shipped_qty

    #update donors table
    cur_donors.loc[dix,'qty'] -= shipped_qty
    don_df.loc[dix,'qty'] -= shipped_qty
    
    #update recipient qty
    cur_recipients.loc[rix,'qty'] -= shipped_qty
    rec_df.loc[rix,'qty'] -= shipped_qty
   
    # this shipping action has one of the following outcomes: (1) brings rrow.qty to 0, (2) brings drow.qty to 0, (3) brings neither to 0

    if rec_df.loc[rix,'qty'] == 0:
        rilocx+=1
        if rilocx == len(rec_df) and totremqty > 0:
            # The decisions is infeasible because I am trying to ship more than requested
            raise('The decisions is infeasible because I am trying to ship more than requested')
    elif don_df.loc[dix,'qty'] == 0:
        dilocx+=1
        if dilocx == len(don_df) and totremqty > 0:
            # The decisions is infeasible because I am trying to ship more than supplied
            raise('The decisions is infeasible because I am trying to ship more than supplied')
    else:
        # should be totremqty == 0
        if totremqty != 0:
            raise('Weird error. If I am here, I should have totremqty == 0')

print ('========================================================')
print(f'cur_donors = \n{cur_donors}\n cur_recipients = \n{cur_recipients}\n')
print(f'remaining qty = {totremqty}. Donor row = [{drow.name,drow.don_id,drow.qty}]. Recipient row = [{rrow.name,rrow.rec_id,rrow.qty}].')

# remove from tables those with qty == 0
cur_donors = cur_donors.loc[cur_donors.qty > 0]
cur_recipients = cur_recipients.loc[cur_recipients.qty > 0]



decision: ship 185 from D1 to R1
cur_donors = 
  don_id                      date    ppe  qty  don_req_id
0     D1 2020-04-09 13:08:00+00:00  gowns  150           0
1     D1 2020-04-09 13:08:00+00:00  gowns  150           1
2     D2 2020-04-10 13:08:00+00:00  gowns  100           2
 cur_recipients = 
  rec_id                      date    ppe  qty
0     R1 2020-04-10 13:08:00+00:00  gowns  100
1     R1 2020-04-10 13:08:00+00:00  gowns  100
2     R2 2020-04-10 13:08:00+00:00  gowns  200

remaining qty = 185. Donor row = [(0, 'D1', 150)]. Recipient row = [(0, 'R1', 100)].
Granular decisions: ship 100 from D1 to R1
cur_donors = 
  don_id                      date    ppe  qty  don_req_id
0     D1 2020-04-09 13:08:00+00:00  gowns   50           0
1     D1 2020-04-09 13:08:00+00:00  gowns  150           1
2     D2 2020-04-10 13:08:00+00:00  gowns  100           2
 cur_recipients = 
  rec_id                      date    ppe  qty
0     R1 2020-04-10 13:08:00+00:00  gowns    0
1     R1 2020-04-1

In [171]:
granular_decisions

,don_id,rec_id,ppe,date,qty,distance,holding time
0,D1,R1,gowns,2020-04-11 13:08:00+00:00,100,-1,2
1,D1,R1,gowns,2020-04-11 13:08:00+00:00,50,-1,2
2,D1,R1,gowns,2020-04-11 13:08:00+00:00,35,-1,2


In [ ]:
for _,dec in decisions.iterrows():
    rem_qty_to_ship = dec.qty
    ppe = dec.ppe
    don = dec.don_id
    don_df = cur_donors[(cur_donors.don_id == don)&(cur_donors.ppe == ppe)]
    for _,row in don_df.iterrows():
        request_idx = row.name
        request_qty = row.qty
        request_date = row.date
        if rem_qty_to_ship < request_qty:
            #  modify the row, but keep it, then exit the loop
            cur_donors.loc[request_idx,'qty'] -= rem_qty_to_ship
            rem_qty_to_ship = 0
        else:
            if rem_qty_to_ship > request_qty:
                print(f'Error: a donor is shipping more {ppe} than available')
            # remove the row and update remo_qty_to_ship
            rem_qty_to_ship -= request_qty
            don_requests_to_remove.append(request_idx)


# Set up result table

In [17]:
result = pd.DataFrame(columns=['metric_name','description','value'])
result

,metric_name,description,value


# Fill rate for rec_id,ppe

In [31]:
total_request = recipients.groupby(['rec_id','ppe'])['qty'].agg(['sum'])
total_request =total_request.reset_index()
total_request.columns=['rec_id','ppe','qty']
total_request

,rec_id,ppe,qty
0,R1,gloves,300.0
1,R1,masks,100.0
2,R2,gloves,200.0
3,R2,gowns,50.0
4,R3,gowns,60.0


In [32]:
total_request.merge(decisions,how='left',on=['rec_id','ppe'],suffixes=['_rec','_dec'])

,rec_id,ppe,qty_rec,date,don_id,qty_dec,distance
0,R1,gloves,300.0,2020-04-15 13:08:00+00:00,D1,300.0,100.0
1,R1,masks,100.0,2020-04-20 13:08:00+00:00,D2,130.0,50.0
2,R2,gloves,200.0,2020-04-15 13:08:00+00:00,D1,100.0,100.0
3,R2,gowns,50.0,2020-04-15 13:08:00+00:00,D1,20.0,100.0
4,R2,gowns,50.0,2020-04-20 13:08:00+00:00,D2,30.0,50.0
5,R3,gowns,60.0,NaT,NaN,NaN,NaN


In [33]:
fr = total_request.merge(decisions,how='left',on=['rec_id','ppe'],suffixes=['_rec','_dec']).groupby(['rec_id','ppe']).agg({'qty_rec':['mean'],'qty_dec':['sum','size']})
fr = fr.reset_index()
fr.columns = ['rec_id','ppe','requested','received','fill_rate']
fr['fill_rate'] = fr['received'] / fr['requested']
fr.loc[fr['fill_rate'] > 1,'fill_rate'] = 1
fr['fill_rate'] = fr['fill_rate'].fillna(0)
fr

,rec_id,ppe,requested,received,fill_rate
0,R1,gloves,300.0,300.0,1.0
1,R1,masks,100.0,130.0,1.0
2,R2,gloves,200.0,100.0,0.5
3,R2,gowns,50.0,50.0,1.0
4,R3,gowns,60.0,0.0,0.0


In [35]:
result['metric_name'] = "fill rate (" + fr['rec_id'] + "," + fr['ppe'] + ")"
result['description'] = "fill rate of recipient " + fr['rec_id'] + " limited to " + fr['ppe']
result['value'] = fr['fill_rate']
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1.0
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1.0
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1.0
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0.0


In [36]:
fr

,rec_id,ppe,requested,received,fill_rate
0,R1,gloves,300.0,300.0,1.0
1,R1,masks,100.0,130.0,1.0
2,R2,gloves,200.0,100.0,0.5
3,R2,gowns,50.0,50.0,1.0
4,R3,gowns,60.0,0.0,0.0


In [38]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1.0
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1.0
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1.0
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0.0


# Fill rate for each ppe

For each ppe, the average fill rate from table fr

In [39]:
fr_p = fr.groupby('ppe')['fill_rate'].mean()
fr_p

ppe
gloves    0.75
gowns     0.50
masks     1.00
Name: fill_rate, dtype: float64

In [40]:
for ppe,val in fr_p.items():
    result.loc[len(result)] = [f'fill rate ({ppe})', f'average fill rate among recipients who requested {ppe}',val]

In [41]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1.00
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1.00
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.50
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1.00
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0.00
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.50
7,fill rate (masks),average fill rate among recipients who request...,1.00


Excluding the zero fill_rates

In [42]:
fr_p_zero = fr[fr.fill_rate > 0].groupby('ppe')['fill_rate'].mean()
fr_p_zero

ppe
gloves    0.75
gowns     1.00
masks     1.00
Name: fill_rate, dtype: float64

In [43]:
for ppe,val in fr_p_zero.items():
    result.loc[len(result)] = [f'fill rate exc zeros ({ppe})', f'average fill rate among recipients who requested {ppe} and received at least one unit',val]
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1.00
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1.00
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.50
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1.00
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0.00
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.50
7,fill rate (masks),average fill rate among recipients who request...,1.00
8,fill rate exc zeros (gloves),average fill rate among recipients who request...,0.75
9,fill rate exc zeros (gowns),average fill rate among recipients who request...,1.00


# overall fill rate

In [44]:
result.loc[len(result)] = [f'fill rate', f'overall fill rate, i.e., the average of the fill rates (ppe)',fr_p.mean()]
result.loc[len(result)] = [f'fill rate exc zeros', f'overall fill rate among recipients who received something, i.e., the average of the fill rates (ppe) among recipients who received at least one unit',fr_p_zero.mean()]



In [45]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1.000000
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1.000000
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.500000
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1.000000
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0.000000
5,fill rate (gloves),average fill rate among recipients who request...,0.750000
6,fill rate (gowns),average fill rate among recipients who request...,0.500000
7,fill rate (masks),average fill rate among recipients who request...,1.000000
8,fill rate exc zeros (gloves),average fill rate among recipients who request...,0.750000
9,fill rate exc zeros (gowns),average fill rate among recipients who request...,1.000000


# Unit-miles

In [18]:
decisions['unit_miles'] = decisions['distance'] * decisions['qty']

gb = decisions.groupby('ppe')
rr = (gb['unit_miles'].sum() / gb['qty'].sum()).to_frame().reset_index()
rr.columns=['ppe','avg_unit_miles']
rr

,ppe,avg_unit_miles
0,gloves,100.0
1,gowns,70.0
2,masks,50.0


In [19]:
for _,row in rr.iterrows():
    ppe = row['ppe']
    result.loc[len(result)] = [f'avg unit-miles ({ppe})', f'average miles travelled by each unit of {ppe}',row['avg_unit_miles']]

In [20]:
overall_unit_miles = decisions.unit_miles.sum() / decisions.qty.sum()
result.loc[len(result)] = [f'avg unit-miles', f'average miles travelled by each unit of ppe',overall_unit_miles]

# Holding time

In [21]:
dd = decisions.merge(donors,on=['don_id','ppe'],suffixes=['_dec','_don'])
dd['holding_time'] = (dd['date_dec'] - dd['date_don']).dt.days
dd['unit_holding_time'] = dd['holding_time'] * dd['qty_dec']
dd

,date_dec,don_id,rec_id,ppe,qty_dec,distance,unit_miles,date_don,qty_don,don_req_id,holding_time,unit_holding_time
0,2020-04-15 13:08:00+00:00,D1,R1,gloves,300,100,30000,2020-04-05 13:08:00+00:00,400,0,10,3000
1,2020-04-15 13:08:00+00:00,D1,R2,gloves,100,100,10000,2020-04-05 13:08:00+00:00,400,0,10,1000
2,2020-04-15 13:08:00+00:00,D1,R2,gowns,20,100,2000,2020-04-11 13:08:00+00:00,20,1,4,80
3,2020-04-20 13:08:00+00:00,D2,R1,masks,130,50,6500,2020-04-12 13:08:00+00:00,130,3,8,1040
4,2020-04-20 13:08:00+00:00,D2,R2,gowns,30,50,1500,2020-04-12 13:08:00+00:00,30,2,8,240


In [22]:
gb = dd.groupby('ppe')
rr = (gb['unit_holding_time'].sum() / gb['qty_dec'].sum()).to_frame().reset_index()
rr.columns=['ppe','avg_unit_days']
rr

,ppe,avg_unit_days
0,gloves,10.0
1,gowns,6.4
2,masks,8.0


In [23]:
for _,row in rr.iterrows():
    ppe = row['ppe']
    result.loc[len(result)] = [f'avg unit-days ({ppe})', f'average days that each unit of {ppe} stayed idle',row['avg_unit_days']]

In [24]:
overall_holding_time = dd.unit_holding_time.sum() / dd.qty_dec.sum()
result.loc[len(result)] = [f'avg holding time', f'average days that each unit of ppe stayed idle',overall_holding_time]

# Average number of shipments among donors

In [25]:
total_shipments = len(decisions.groupby(['don_id','rec_id']).size())
donors = decisions['don_id'].nunique()
result.loc[len(result)] = [f'avg number of shipments', f'average number of shipments among donors',total_shipments/donors]

In [26]:
result

,metric_name,description,value
0,"fill rate (R1,gloves)",fill rate of recipient R1 limited to gloves,1
1,"fill rate (R1,masks)",fill rate of recipient R1 limited to masks,1
2,"fill rate (R2,gloves)",fill rate of recipient R2 limited to gloves,0.5
3,"fill rate (R2,gowns)",fill rate of recipient R2 limited to gowns,1
4,"fill rate (R3,gowns)",fill rate of recipient R3 limited to gowns,0
5,fill rate (gloves),average fill rate among recipients who request...,0.75
6,fill rate (gowns),average fill rate among recipients who request...,0.5
7,fill rate (masks),average fill rate among recipients who request...,1
8,fill rate exc zeros (gloves),average fill rate among recipients who request...,0.75
9,fill rate exc zeros (gowns),average fill rate among recipients who request...,1
